<a href="https://colab.research.google.com/github/mavihsrr/GraphRAG-Investigation/blob/main/kG_unknown_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
from langchain.schema import Document

In [3]:

input_txt_file = Path("/content/crime.txt")


outputdirectory = Path("./output")
with open(input_txt_file, 'r', encoding='utf-8') as file:
    document_content = file.read()


documents = [Document(page_content=document_content, metadata={})]

# Text Splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

# Split the single document into chunks
pages = splitter.split_documents(documents)

In [4]:
from langchain_community.chat_models import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate

llm = ChatPerplexity(
    temperature=0, pplx_api_key="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", model="llama-3.1-sonar-small-128k-online"
)
system = "given is an investigation report that is unsolved. being an expert in researching, look up the given case that in the passed input online and collect as much data as you could about it. make sure everything is relevant and that there is no bias content. only add important stuff that you feel is missing."
human = "{input}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm
response = chain.invoke({"input": ""})
response.content

BadRequestError: Error code: 400 - {'error': {'message': 'Message content was empty', 'type': 'invalid_message', 'code': 400}}

In [ ]:
from pathlib import Path
from langchain_community.chat_models import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter


llm = ChatPerplexity(
    temperature=0, pplx_api_key="pplx-",
    model="llama-3.1-sonar-small-128k-online"
)

system = "given is an investigation report that is unsolved. being an expert in researching, look up the given case that is in the passed input online and collect as much data as you could about it. make sure everything is relevant and that there is no biased content. only add important stuff that you feel is missing."
human = "{input}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

def process_chunk(chunk):

    chain = prompt | llm
    response = chain.invoke({"input": chunk})

    return response.content if response else ""

def process_chunks(pages):
    updated_pages = []

    for i, chunk in enumerate(pages):
        print(f"Processing chunk {i + 1}/{len(pages)}...")

        chunk_content = chunk.page_content

        additional_info = process_chunk(chunk_content)

        updated_chunk = chunk_content + "\n\n--- Additional Information Gathered ---\n\n" + additional_info

        updated_pages.append(updated_chunk)

    return updated_pages

def save_updated_chunks(pages, output_path):
    output_file = output_path / "enhanced_report.txt"

    with open(output_file, 'w', encoding='utf-8') as file:
        for i, chunk in enumerate(pages):
            file.write(f"--- Chunk {i + 1} ---\n\n")
            file.write(chunk)
            file.write("\n\n")

    print(f"Enhanced report saved to {output_file}")

def augment_report(input_txt_file, output_directory):
    with open(input_txt_file, 'r', encoding='utf-8') as file:
        document_content = file.read()

    documents = [Document(page_content=document_content, metadata={})]

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=150,
        length_function=len,
        is_separator_regex=False,
    )

    pages = splitter.split_documents(documents)

    updated_pages = process_chunks(pages)

    save_updated_chunks(updated_pages, output_directory)

input_txt_file = Path("/content/crime.txt")
output_directory = Path("/content/output")
augment_report(input_txt_file, output_directory)


In [5]:
print(pages[0].page_content)

The Travel Channel featured a new episode of the series Mysteries at the Museum devoted to the Zodiac case. An episode synopsis posted at the Travel Channel website read: “Don Wildman searches for the identity of one of America’s most notorious serial killers. He meets with a retired inspector who worked on the case, learns code breaking from an expert cryptographer and reveals a new theory on who may be responsible.”

MATM-Oranchak
Don Wildman and cipher expert David Oranchak.

Host Don Wildman presented a basic overview of the Zodiac story and visited the four crime scenes. He also consulted with cipher expert David Oranchak. Wildman and Oranchak examined the Zodiac’s first coded message and the still-unsolved 340 cipher while discussing some of the killer’s methods as a cryptographer. 

MATM-Oranchak-2

During his tour of the Zodiac crime scenes, Wildman was accompanied by theorist Mark Hewitt, who also shared his theory that the Zodiac was Ted Kaczynski, the infamous “Unabomber” re

In [6]:
import sys
sys.path.append('/content/helpers')

from helpers.df_helpers import documents2Dataframe


In [7]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(4, 2)


,text,chunk_id
0,The Travel Channel featured a new episode of t...,ddfdfcb0eb0b4cc580f1c29702e6b245
1,MATM-Hewitt\nZodiac theorist Mark Hewitt and h...,43fd249357b34f3caee1cb814a8c85d3
2,MATM-Repetto\nRetired SFPD Inspector Vincent R...,c55f785a4df149d18f4351235b679ef1
3,MATM-DNA\nGary Harmor demonstrates a method us...,6cd0dff9f06d43a1a937c9d23195370e


In [8]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

In [12]:
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='gpt-3.5-turbo-0125')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)

    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 10
## Increasing the weight of the relation to 4.
## We will assign the weight of 1 when later the contextual proximity will be calculated.
print(dfg1.shape)
dfg1.head()

(29, 5)


,node_1,node_2,edge,chunk_id,count
0,don wildman,zodiac case,Don Wildman featured in the Mysteries at the M...,ddfdfcb0eb0b4cc580f1c29702e6b245,10
1,don wildman,david oranchak,Don Wildman consulted with cipher expert David...,ddfdfcb0eb0b4cc580f1c29702e6b245,10
2,david oranchak,zodiac case,David Oranchak examined the Zodiac's coded mes...,ddfdfcb0eb0b4cc580f1c29702e6b245,10
3,don wildman,mark hewitt,Don Wildman was accompanied by theorist Mark H...,ddfdfcb0eb0b4cc580f1c29702e6b245,10
4,mark hewitt,zodiac case,Mark Hewitt shared his theory that the Zodiac ...,ddfdfcb0eb0b4cc580f1c29702e6b245,10


In [1]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

NameError: name 'pd' is not defined

In [13]:
dfg = dfg1
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,arthur leigh allen,ted kaczynski,43fd249357b34f3caee1cb814a8c85d3,Arthur Leigh Allen and Ted Kaczynski are suspe...,10
1,arthur leigh allen,zodiac,43fd249357b34f3caee1cb814a8c85d3,There is a theory that the Zodiac was Arthur L...,10
2,david oranchak,zodiac case,ddfdfcb0eb0b4cc580f1c29702e6b245,David Oranchak examined the Zodiac's coded mes...,10
3,dna,envelopes,c55f785a4df149d18f4351235b679ef1,DNA can be retrieved from old envelopes using ...,10
4,dna,stamps,c55f785a4df149d18f4351235b679ef1,DNA can be retrieved from old stamps using new...,10
5,dna,stamps and envelopes,6cd0dff9f06d43a1a937c9d23195370e,DNA obtained from old stamps and envelopes,10
6,dna,state lab,6cd0dff9f06d43a1a937c9d23195370e,DNA collected in a state lab in May 2018,10
7,dna,usable dna evidence,6cd0dff9f06d43a1a937c9d23195370e,Lab retrieving usable DNA evidence for forensi...,10
8,don wildman,david oranchak,ddfdfcb0eb0b4cc580f1c29702e6b245,Don Wildman consulted with cipher expert David...,10
9,don wildman,mark hewitt,ddfdfcb0eb0b4cc580f1c29702e6b245,Don Wildman was accompanied by theorist Mark H...,10


In [14]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(26,)

In [15]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [16]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  5
[['arthur leigh allen', 'attack at lake berryessa', 'costume', 'mysteries at the museum', 'ted kaczynski', 'zodiac'], ['david oranchak', 'don wildman', 'mark hewitt', 'sfpd inspector', 'vincent repetto', 'zodiac case'], ['dna', 'envelopes', 'forensic process', 'gary harmor', 'genealogical lab', 'serological research institute', 'stamps', 'stamps and envelopes', 'state lab', 'usable dna evidence'], ['douglas oswell', 'michael rusconi'], ['forensic genealogy research', 'identify the zodiac']]


In [17]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,arthur leigh allen,#db5f57,1
1,attack at lake berryessa,#db5f57,1
2,costume,#db5f57,1
3,mysteries at the museum,#db5f57,1
4,ted kaczynski,#db5f57,1
5,zodiac,#db5f57,1
6,david oranchak,#57db94,2
7,don wildman,#57db94,2
8,mark hewitt,#57db94,2
9,sfpd inspector,#57db94,2


In [18]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [21]:
from pyvis.network import Network

graph_output_directory = "/content/output/graph.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

/content/output/graph.html


In [20]:
pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.1 MB/s eta 0:00:00
